modeWeightAnalysis V02

Version Discription: minor modification for latest analysis

Last Update: 20241109

### Basic Setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%run '/content/drive/My Drive/Colab Notebooks/StrokeAI/ClassifierModel_V06_Simplified.ipynb'

Mounted at /content/drive


In [ ]:
import torch
import numpy as np

In [ ]:
# Load the model
model = torch.load("/content/drive/My Drive/Colab Notebooks/StrokeAI/20241109_1430_model.pth")
model.eval()

<ipython-input-3-ffa14cbe3981>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/content/drive/My Drive/Colab Notebooks/StrokeAI/20241109_1430_model.pth"

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 8, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(32, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU

### Weight Analysis

In [ ]:
import pickle
from torchvision import transforms

# Load the test dataset
with open('/content/drive/My Drive/Colab Notebooks/StrokeAI/20241015_2301_test_dataset.pkl', 'rb') as dataset:
  dataset = pickle.load(dataset)

# Placeholder for accumulating tensors
accumulated_gradients = []

# For loop the dataset, calculate gradients for all the inputs
for data in dataset:  # For each data, data[0] is filename/index, data[1] contains tensor for analysis
  input_data = data[1]

  input_tensor = input_data.unsqueeze(0)
  input_tensor.requires_grad = True

  output = model(input_tensor)

  target_class = output.argmax()

  model.zero_grad()  # Zero out any existing gradients
  target = output[0, target_class]
  target.backward()  # Compute gradients

  gradients = input_tensor.grad[0]

  accumulated_gradients.append(gradients)

# Concatenate all gradients along a new dimension and convert list of tensors to a single tensor
accumulated_gradients_tensor = torch.cat(accumulated_gradients, dim=0)

### Statistical Results

In [ ]:
# print(f'There gradients tensor is in shape of {accumulated_gradients_tensor.size()}')
print(f'The max value of all the gradient is {accumulated_gradients_tensor.max()}')
print(f'The min value of all the gradient is {accumulated_gradients_tensor.min()}')

# Calculate the 1st and 99th percentiles of all the gradients
print(f'The 0.03 percentile of all the gradient is {torch.quantile(accumulated_gradients_tensor, 0.003)}')
print(f'The 99.7 percentile of all the gradient is {torch.quantile(accumulated_gradients_tensor, 0.997)}')

# Absolute gradients
abs_accumulated_gradients_tensor = accumulated_gradients_tensor.abs()
print(f'The max value of all absolute gradient is {abs_accumulated_gradients_tensor.max()}')
print(f'The 0.03 percentile of all absolute gradient is {torch.quantile(abs_accumulated_gradients_tensor, 0.003)}')
print(f'The 99.7 percentile of all absolute gradient is {torch.quantile(abs_accumulated_gradients_tensor, 0.997)}')

The max value of all the gradient is 33.060359954833984
The min value of all the gradient is -33.895843505859375
The 0.03 percentile of all the gradient is -3.99369478225708
The 99.7 percentile of all the gradient is 3.9788482189178467
The max value of all absolute gradient is 33.895843505859375
The 0.03 percentile of all absolute gradient is 0.0
The 99.7 percentile of all absolute gradient is 5.065016746520996
